In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.1 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=a26de46271d89a84dc4b53e18f7f6d1365b5f74ef6cb3f79575a238b5b4e26ad
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [ ]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.4 MB/s 


In [1]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [3]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [4]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0530.csv	    dataframe_result_530_UK.csv
 a0531.csv	    dataframe_result_531_UK.csv
 a0601.csv	    dataframe_result_csv
 a0602.csv	    dataframe_result_csv_EU
 a0603.csv	    dataframe_result_csv_UK
 a0604.csv	    Github
 a0605.csv	    heatmap529_UK.ipynb
 a0606.csv	    heatmap530.ipynb
 a0607.csv	    Heat_Map_Online_language.ipynb
 a0608.csv	    __MACOSX
 a0609.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [9]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0529.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_529_UK.csv
 a0530.csv	    dataframe_result_530_UK.csv
 a0531.csv	    dataframe_result_531_UK.csv
 a0601.csv	    dataframe_result_csv
 a0602.csv	    dataframe_result_csv_EU
 a0603.csv	    dataframe_result_csv_UK
 a0604.csv	    Github
 a0605.csv	    heatmap529_UK.ipynb
 a0606.csv	    heatmap530.ipynb
 a0607.csv	    Heat_Map_Online_language.ipynb
 a0608.csv	    __MACOSX
 a0609.csv	    mapping_UK
 a0610.csv	    mapping_US
 a0611.csv	    non_violant1.csv
 ccc_compiled.csv   non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [ ]:
###
#ct=pd.read_csv('a0601.csv')

In [10]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,8475622807_10158307491652808,Facebook,2020-05-29 23:59:58,2021-07-18 22:02:59,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/17u7bHE,https://www.facebook.com/8475622807/posts/1015...,531831,1.389908,1.0,76,26,105,7,2,1,20,66,0,2,49,30,55,8,12,12,41,11,0,6,11932,Global BC,GlobalBC,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to the Global BC Facebook page\n\n\n,True,2008-02-19 17:12:23,en,0,11932|10158307491652808,{},{},{}
1,1,62571159864_10157572809069865,Facebook,2020-05-29 23:59:58,2021-06-30 22:52:44,link,"Chauvin arrested, charged; Walz says more enfo...",bizjournals.com,,State leaders hope that Friday night won't see...,1.0,https://www.bizjournals.com/twincities/news/20...,https://www.facebook.com/62571159864/posts/101...,11941,-5.750000,1.0,3,0,0,0,0,1,0,0,0,0,3,2,3,2,2,4,4,3,0,1,26543,Minneapolis-St Paul Business Journal,mspbj,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Subscribe today!\nhttps://bit.ly/33ZDnTt,True,2009-02-10 03:06:59,en,0,26543|10157572809069865,"{'Minneapolis': 1, 'St. Paul': 1}",{},{}
2,2,133713529984442_3343833268972436,Facebook,2020-05-29 23:59:57,2021-07-06 15:03:56,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/uxuoPTh,https://www.facebook.com/133713529984442/posts...,58753,-1.500000,1.0,9,3,7,1,0,0,7,11,0,0,6,6,8,10,4,6,11,6,0,4,134502,CHBC - Global Okanagan,globalokanagan,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,Welcome to the Global Okanagan page on Faceboo...,True,2010-07-02 16:29:04,en,0,134502|3343833268972436,{},{},{}
3,3,10080840973_10157055476960974,Facebook,2020-05-29 23:59:57,2021-07-19 07:54:29,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/S8uJhZ5,https://www.facebook.com/10080840973/posts/101...,228578,-1.146341,1.0,23,0,6,0,0,0,7,5,0,0,7,4,7,3,3,6,8,9,0,3,11933,Global National,GlobalNational,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,GLOBAL NATIONAL WITH DAWNA FRIESEN is Canada's...,True,2008-02-16 23:35:25,en,0,11933|10157055476960974,{},{},{}
4,4,233680270017312_3274095189309123,Facebook,2020-05-29 23:59:56,2021-09-03 06:07:38,link,Etalk host makes emotional plea for change,calgary.ctvnews.ca,Floyd gradually grew unresponsive and ceased t...,Etalk host Tyrone Edwards made an emotional pl...,1.0,https://calgary.ctvnews.ca/video?cid=sm%3Atrue...,https://www.facebook.com/233680270017312/posts...,63820,-6.727273,1.0,3,2,1,1,0,1,2,1,0,0,13,6,18,4,3,9,9,12,0,2,422904,CTV Calgary,CTVCalgary,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to CTV Calgary's home on Facebook. CTV...,True,2011-09-16 17:13:46,en,0,422904|3274095189309123,{'Minneapolis': 1},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [11]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [ ]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [12]:
###
ct_ner = ct

In [13]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},{}
1,"{'Minneapolis': 1, 'St. Paul': 1}",{},{}
2,{},{},{}
3,{},{},{}
4,{'Minneapolis': 1},{},{}


In [14]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [15]:
ct_ner['NER-msg']

0                             []
1        [St. Paul, Minneapolis]
2                             []
3                             []
4                  [Minneapolis]
                  ...           
39755                         []
39756              [Minneapolis]
39757                [Minnesota]
39758                         []
39759                         []
Name: NER-msg, Length: 39760, dtype: object

In [16]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [17]:
ct_ner['NER']

0                                                       []
1                                  [St. Paul, Minneapolis]
2                                                       []
3                                                       []
4                                            [Minneapolis]
                               ...                        
39755                                        [Minneapolis]
39756                                        [Minneapolis]
39757    [Minnesota, America, Minnesota, the United Sta...
39758          [America, Minnesota, the United States, US]
39759          [America, Minnesota, the United States, US]
Name: NER, Length: 39760, dtype: object

In [18]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [20]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,8475622807_10158307491652808,Facebook,2020-05-29 23:59:58,2021-07-18 22:02:59,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/17u7bHE,https://www.facebook.com/8475622807/posts/1015...,531831,1.389908,1.0,76,26,105,7,2,1,20,66,0,2,49,30,55,8,12,12,41,11,0,6,11932,Global BC,GlobalBC,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to the Global BC Facebook page\n\n\n,True,2008-02-19 17:12:23,en,0,11932|10158307491652808,[],[],[],[]
1,1,62571159864_10157572809069865,Facebook,2020-05-29 23:59:58,2021-06-30 22:52:44,link,"Chauvin arrested, charged; Walz says more enfo...",bizjournals.com,,State leaders hope that Friday night won't see...,1.0,https://www.bizjournals.com/twincities/news/20...,https://www.facebook.com/62571159864/posts/101...,11941,-5.750000,1.0,3,0,0,0,0,1,0,0,0,0,3,2,3,2,2,4,4,3,0,1,26543,Minneapolis-St Paul Business Journal,mspbj,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Subscribe today!\nhttps://bit.ly/33ZDnTt,True,2009-02-10 03:06:59,en,0,26543|10157572809069865,"[St. Paul, Minneapolis]",[],[],"[st. paul, minneapolis]"
2,2,133713529984442_3343833268972436,Facebook,2020-05-29 23:59:57,2021-07-06 15:03:56,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/uxuoPTh,https://www.facebook.com/133713529984442/posts...,58753,-1.500000,1.0,9,3,7,1,0,0,7,11,0,0,6,6,8,10,4,6,11,6,0,4,134502,CHBC - Global Okanagan,globalokanagan,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,Welcome to the Global Okanagan page on Faceboo...,True,2010-07-02 16:29:04,en,0,134502|3343833268972436,[],[],[],[]
3,3,10080840973_10157055476960974,Facebook,2020-05-29 23:59:57,2021-07-19 07:54:29,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/S8uJhZ5,https://www.facebook.com/10080840973/posts/101...,228578,-1.146341,1.0,23,0,6,0,0,0,7,5,0,0,7,4,7,3,3,6,8,9,0,3,11933,Global National,GlobalNational,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,GLOBAL NATIONAL WITH DAWNA FRIESEN is Canada's...,True,2008-02-16 23:35:25,en,0,11933|10157055476960974,[],[],[],[]
4,4,233680270017312_3274095189309123,Facebook,2020-05-29 23:59:56,2021-09-03 06:07:38,link,Etalk host makes emotional plea for change,calgary.ctvnews.ca,Floyd gradually grew unresponsive and ceased t...,Etalk host Tyrone Edwards made an emotional pl...,1.0,https://calgary.ctvnews.ca/video?cid=sm%3Atrue...,https://www.facebook.com/233680270017312/posts...,63820,-6.727273,1.0,3,2,1,1,0,1,2,1,0,0,13,6,18,4,3,9,9,12,0,2,422904,CTV Calgary,CTVCalgary,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to CTV Calgary's home on Facebook. CTV...,True,2011-09-16 17:13:46,en,0,422904|3274095189309123,[Minneapolis],[],[],[minneapolis]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [21]:
ct_ner['NER-1']=ct_ner['NER']

In [24]:
ct_ner['NER-1'][1]

['st. paul', 'minneapolis']

In [25]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 12.2 MB/s 


In [26]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [27]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,8475622807_10158307491652808,Facebook,2020-05-29 23:59:58,2021-07-18 22:02:59,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/17u7bHE,https://www.facebook.com/8475622807/posts/1015...,531831,1.389908,1.0,76,26,105,7,2,1,20,66,0,2,49,30,55,8,12,12,41,11,0,6,11932,Global BC,GlobalBC,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to the Global BC Facebook page\n\n\n,True,2008-02-19 17:12:23,en,0,11932|10158307491652808,[],[],[],[],[]
1,1,62571159864_10157572809069865,Facebook,2020-05-29 23:59:58,2021-06-30 22:52:44,link,"Chauvin arrested, charged; Walz says more enfo...",bizjournals.com,,State leaders hope that Friday night won't see...,1.0,https://www.bizjournals.com/twincities/news/20...,https://www.facebook.com/62571159864/posts/101...,11941,-5.750000,1.0,3,0,0,0,0,1,0,0,0,0,3,2,3,2,2,4,4,3,0,1,26543,Minneapolis-St Paul Business Journal,mspbj,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Subscribe today!\nhttps://bit.ly/33ZDnTt,True,2009-02-10 03:06:59,en,0,26543|10157572809069865,"[St. Paul, Minneapolis]",[],[],"[st. paul, minneapolis]","[st. paul, minneapolis]"
2,2,133713529984442_3343833268972436,Facebook,2020-05-29 23:59:57,2021-07-06 15:03:56,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/uxuoPTh,https://www.facebook.com/133713529984442/posts...,58753,-1.500000,1.0,9,3,7,1,0,0,7,11,0,0,6,6,8,10,4,6,11,6,0,4,134502,CHBC - Global Okanagan,globalokanagan,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,Welcome to the Global Okanagan page on Faceboo...,True,2010-07-02 16:29:04,en,0,134502|3343833268972436,[],[],[],[],[]
3,3,10080840973_10157055476960974,Facebook,2020-05-29 23:59:57,2021-07-19 07:54:29,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/S8uJhZ5,https://www.facebook.com/10080840973/posts/101...,228578,-1.146341,1.0,23,0,6,0,0,0,7,5,0,0,7,4,7,3,3,6,8,9,0,3,11933,Global National,GlobalNational,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,GLOBAL NATIONAL WITH DAWNA FRIESEN is Canada's...,True,2008-02-16 23:35:25,en,0,11933|10157055476960974,[],[],[],[],[]
4,4,233680270017312_3274095189309123,Facebook,2020-05-29 23:59:56,2021-09-03 06:07:38,link,Etalk host makes emotional plea for change,calgary.ctvnews.ca,Floyd gradually grew unresponsive and ceased t...,Etalk host Tyrone Edwards made an emotional pl...,1.0,https://calgary.ctvnews.ca/video?cid=sm%3Atrue...,https://www.facebook.com/233680270017312/posts...,63820,-6.727273,1.0,3,2,1,1,0,1,2,1,0,0,13,6,18,4,3,9,9,12,0,2,422904,CTV Calgary,CTVCalgary,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to CTV Calgary's home on Facebook. CTV...,True,2011-09-16 17:13:46,en,0,422904|3274095189309123,[Minneapolis],[],[],[minneapolis],[minneapolis]
...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [28]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [30]:
!ls

dataframe_mapping_0529.csv


In [31]:
ct_mapping=pd.read_csv('dataframe_mapping_0529.csv',encoding='ISO-8859-1')

In [32]:
len(ct_mapping['City'])

2349

In [33]:
len(ct_ner['NER-1'])

39760

In [34]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [35]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,8475622807_10158307491652808,Facebook,2020-05-29 23:59:58,2021-07-18 22:02:59,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/17u7bHE,https://www.facebook.com/8475622807/posts/1015...,531831,1.389908,1.0,76,26,105,7,2,1,20,66,0,2,49,30,55,8,12,12,41,11,0,6,11932,Global BC,GlobalBC,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to the Global BC Facebook page\n\n\n,True,2008-02-19 17:12:23,en,0,11932|10158307491652808,[],[],[],[],[]
1,1,62571159864_10157572809069865,Facebook,2020-05-29 23:59:58,2021-06-30 22:52:44,link,"Chauvin arrested, charged; Walz says more enfo...",bizjournals.com,,State leaders hope that Friday night won't see...,1.0,https://www.bizjournals.com/twincities/news/20...,https://www.facebook.com/62571159864/posts/101...,11941,-5.750000,1.0,3,0,0,0,0,1,0,0,0,0,3,2,3,2,2,4,4,3,0,1,26543,Minneapolis-St Paul Business Journal,mspbj,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Subscribe today!\nhttps://bit.ly/33ZDnTt,True,2009-02-10 03:06:59,en,0,26543|10157572809069865,"[St. Paul, Minneapolis]",[],[],"[st. paul, minneapolis]","[Saint Paul, Ramsey County, Minnesota, United ..."
2,2,133713529984442_3343833268972436,Facebook,2020-05-29 23:59:57,2021-07-06 15:03:56,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/uxuoPTh,https://www.facebook.com/133713529984442/posts...,58753,-1.500000,1.0,9,3,7,1,0,0,7,11,0,0,6,6,8,10,4,6,11,6,0,4,134502,CHBC - Global Okanagan,globalokanagan,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,Welcome to the Global Okanagan page on Faceboo...,True,2010-07-02 16:29:04,en,0,134502|3343833268972436,[],[],[],[],[]
3,3,10080840973_10157055476960974,Facebook,2020-05-29 23:59:57,2021-07-19 07:54:29,link,Officer charged with murder in George Floydâ...,globalnews.ca,Floyd gradually grew unresponsive and ceased t...,"âWe have evidence, we have the citizenâs c...",2.0,https://trib.al/S8uJhZ5,https://www.facebook.com/10080840973/posts/101...,228578,-1.146341,1.0,23,0,6,0,0,0,7,5,0,0,7,4,7,3,3,6,8,9,0,3,11933,Global National,GlobalNational,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_page,CA,GLOBAL NATIONAL WITH DAWNA FRIESEN is Canada's...,True,2008-02-16 23:35:25,en,0,11933|10157055476960974,[],[],[],[],[]
4,4,233680270017312_3274095189309123,Facebook,2020-05-29 23:59:56,2021-09-03 06:07:38,link,Etalk host makes emotional plea for change,calgary.ctvnews.ca,Floyd gradually grew unresponsive and ceased t...,Etalk host Tyrone Edwards made an emotional pl...,1.0,https://calgary.ctvnews.ca/video?cid=sm%3Atrue...,https://www.facebook.com/233680270017312/posts...,63820,-6.727273,1.0,3,2,1,1,0,1,2,1,0,0,13,6,18,4,3,9,9,12,0,2,422904,CTV Calgary,CTVCalgary,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,Welcome to CTV Calgary's home on Facebook. CTV...,True,2011-09-16 17:13:46,en,0,422904|3274095189309123,[Minneapolis],[],[],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, 

In [36]:
ct_ner['NER-1'][12]

['San Jose, Santa Clara County, California, United States']

In [37]:
ct_ner.to_csv('dataframe0611_post.csv')